# Normal Linear Regression Model

Classic linear regression model:

$y_i=x_i'\beta+\varepsilon_i$

Assumptions:
1. $\varepsilon_i \overset{iid}{\sim} N(0,\sigma^2)$
2. $x_i \perp \varepsilon_j$

$\Rightarrow f(y_i \mid x_i,\beta,\sigma^2) \sim N(x_i\beta,\sigma^2)$ is the likelihood function



Since $\varepsilon_i$ and $\varepsilon_j$ are independent $\forall i \neq j$ and $y-X\beta = y-X(\beta-\hat{\beta})-X\hat{\beta} $:


$f(y \mid x,\beta,\sigma^2) = \frac{1}{(2\pi)^{n/2}\sigma^{n-\nu}}exp \left(-\frac{1}{2\sigma^2}(\beta-\hat{\beta})'X'X(\beta-\hat{\beta}) \right) \frac{1}{\sigma^\nu}exp \left(-\frac{s^2\nu}{2\sigma^2}\right)$

where

$\hat{\beta}=(X'X)^{-1}X'y$

$s^2=\frac{(y-X\hat{\beta})'(y-X\hat{\beta})}{\nu}$

$\nu=n-k$



In [1]:
import numpy as np
import pandas as pd

# Generate sample
n = 100

trueBeta = np.array([10,5])

X = pd.DataFrame(index=range(n))
X['x0']=np.ones(n)
X['x1']=np.random.randn(n)
# X['x2']=np.random.randn(n,1)*5+10
# X['x3']=np.random.randn(n,1)*2-20

y = pd.DataFrame(index=range(n))
y['y']=np.dot(X,trueBeta)+np.random.randn(n)


## Choosing the prior
Memo: prior is Normal-Inverted Gamma

$$(\beta,\sigma^2) \sim NIG \left(\underline{\beta},\underline{V},\frac{1}{\underline{\sigma^2}},\underline{\nu} \right)$$


We can interpret the prior as

$\pi(\beta,\sigma^2) = \pi_\beta(\beta \mid \sigma^2) \times \pi_\sigma(\sigma^2)$

$\beta \mid \sigma^2 \sim N(\beta_0,\sigma^2V_0)$

$\sigma^2 \sim IG \left(\frac{1}{\sigma_0^2},\nu_0 \right)$

That is, draw $\sigma^2$ from IG, then draw $\beta$ from N

In [2]:
# Choosing the prior

m = 1 #confidence in prior. The larger this number, the smaller the prior variance

prior = {'b0': np.array([[9],
                         [6]]),
         'V0': 0.05/m * np.identity(2),
         'sigma2_0':1,
         'nu0':m }

In [3]:
print(prior['V0'])
print(prior['b0'])
print(prior['sigma2_0'])
print(prior['nu0'])

[[ 0.05  0.  ]
 [ 0.    0.05]]
[[9]
 [6]]
1
1


## Calculating the posterior
Normal-Inverted Gamma is a conjugate prior, i.e. yields a Normal-Inverted Gamma posterior (with different hyperparameters)
$$(\beta,\sigma^2 \mid y,X) \sim NIG \left(\overline{\beta},\overline{V},\frac{1}{\overline{\sigma^2}},\overline{\nu} \right)$$

where

$\overline{V} = \left( \underline{V}^{-1}+X'X \right)^{-1}$

$\overline{\beta} = \overline{V} \left( \underline{V}^{-1}\underline{\beta}+X'X\hat{\beta} \right)$

$\overline{\nu} = \underline{\nu}+n$

$\overline{\sigma}^2 = \frac{1}{\overline{\nu}} \left(\underline{\nu \sigma}^2 +(n-k)s^2+ (\hat{\beta}-\underline{\beta})'(\underline{V}+(X'X)^{-1})(\hat{\beta}-\underline{\beta}) \right) $

In [4]:
class NLRM(object):
    def __init__(self,Y,X,prior):
        self.g(Y,X,prior)
    def g(self,Y,X,prior):
        # OLS beta estimator
        b_hat = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X),X)),np.transpose(X)),y)
        
        n = X.shape[0]
        k = X.shape[1]
        
        s2 = np.dot(np.transpose(y-np.dot(X,b_hat)),y-np.dot(X,b_hat))/(n-k)
        
        V = np.linalg.inv(np.linalg.inv(prior['V0'])+np.dot(np.transpose(X),X))
        
        b = np.dot(V,(np.dot(np.linalg.inv(prior['V0']),prior['b0'])+np.dot(np.dot(np.transpose(X),X),b_hat)))
        
        nu = prior['nu0'] + n
        sigma2 = 1/nu * (prior['nu0']*prior['sigma2_0'] + (n-k)*s2 + 
                        np.dot(np.dot(np.transpose(b_hat-b),
                                      prior['V0']+np.linalg.inv(np.dot(np.transpose(X),X))),
                              b_hat-b))
        
        self.b_hat = b_hat
        self.s2 = s2
        self.V = V
        self.b = b
        self.nu = nu
        self.sigma2 = sigma2

### test
model = NLRM(y,X,prior)

print('beta:')
print(model.b)
print('')
print('V:')
print(model.V)
print('')
print('nu: '+str(model.nu))
print('')
print('sigma2: '+str(model.sigma2))

beta:
[[ 9.81085477]
 [ 5.22525282]]

V:
[[ 0.00862388 -0.00164656]
 [-0.00164656  0.00933139]]

nu: 101

sigma2: [[ 1.17337512]]
